<a href="https://colab.research.google.com/github/GzpTez0514/-/blob/main/Pytorch%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A010_%E6%9E%84%E7%AD%91%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 卷积层
import torch
import torch.nn as nn

data = torch.ones(size=(10, 3, 28, 28)) # 10张尺寸为28*28的，拥有3个通道的图像
conv1 = nn.Conv2d(in_channels=3, 
          out_channels=6, # 全部通道的扫描值被合并，6个卷积核形成6个feature map
          kernel_size=3 # 表示3x3的卷积核
          )

conv2 = nn.Conv2d(in_channels=6, # 对下一层网络来说，输入的是上层生成的6个feature map
          out_channels=4, # 全部特征图的扫描值被合并，4个卷积核形成4个新的feature map
          kernel_size=3)

print(conv1(data).shape)
print(conv2(conv1(data)).shape)


torch.Size([10, 6, 26, 26])
torch.Size([10, 4, 24, 24])


In [ ]:
# 特征图的尺寸 （H + 2P -K）/ S + 1
data = torch.ones(size=(10, 3, 28, 28))
conv1 = nn.Conv2d(3, 6, 3)
conv2 = nn.Conv2d(6, 4, 3)
conv3 = nn.Conv2d(4, 16, 5, stride=2, padding=1) 
conv4 = nn.Conv2d(16, 3, 5, stride=3, padding=2)


In [ ]:
# VGG16的复现
# (卷积x2 + 池化) x2 -> (卷积x3 + 池化) x3 —> FC层x3 每组卷积+池化算一个block
!pip install torchinfo
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchinfo import summary

class VGG16(nn.Module):
  def __init__(self):
    super().__init__()
    # block1
    self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
    self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
    self.pool1 = nn.MaxPool2d(2, 2)

    # block2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
    self.pool2 = nn.MaxPool2d(2, 2)

    # block3
    self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
    self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
    self.conv7 = nn.Conv2d(256, 256, 3, padding=1)
    self.pool3 = nn.MaxPool2d(2, 2)

    # block4
    self.conv8 = nn.Conv2d(256, 512, 3, padding=1)
    self.conv9 = nn.Conv2d(512, 512, 3, padding=1)
    self.conv10 = nn.Conv2d(512, 512, 3, padding=1)
    self.pool4 = nn.MaxPool2d(2, 2)

    # block5
    self.conv11 = nn.Conv2d(512, 512, 3, padding=1)
    self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
    self.conv13 = nn.Conv2d(512, 512, 3, padding=1)
    self.pool5 = nn.MaxPool2d(2,2)

    # FC层
    self.linear1 = nn.Linear(512*7*7, 4096)
    self.linear2 = nn.Linear(4096, 4096)
    self.linear3 = nn.Linear(4096, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.pool1(x)

    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = self.pool2(x)

    x = F.relu(self.conv5(x))
    x = F.relu(self.conv6(x))
    x = F.relu(self.conv7(x))
    x = self.pool3(x)

    x = F.relu(self.conv8(x))
    x = F.relu(self.conv9(x))
    x = F.relu(self.conv10(x))
    x = self.pool4(x)

    x = F.relu(self.conv11(x))
    x = F.relu(self.conv12(x))
    x = F.relu(self.conv13(x))
    x = self.pool5(x)
    
    x = x.view(-1, 512*7*7)

    x = F.relu(self.linear1(F.dropout(x, p=0.5)))
    x = F.relu(self.linear2(F.dropout(x, p=0.5)))

    output = F.softmax(self.linear3(x), dim=1)
    return output

vgg = VGG16()
summary(vgg, input_size=(10, 3, 224, 224), device='cpu')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [10, 10]                  --
├─Conv2d: 1-1                            [10, 64, 224, 224]        1,792
├─Conv2d: 1-2                            [10, 64, 224, 224]        36,928
├─MaxPool2d: 1-3                         [10, 64, 112, 112]        --
├─Conv2d: 1-4                            [10, 128, 112, 112]       73,856
├─Conv2d: 1-5                            [10, 128, 112, 112]       147,584
├─MaxPool2d: 1-6                         [10, 128, 56, 56]         --
├─Conv2d: 1-7                            [10, 256, 56, 56]         295,168
├─Conv2d: 1-8                            [10, 256, 56, 56]         590,080
├─Conv2d: 1-9                            [10, 256, 56, 56]         590,080
├─MaxPool2d: 1-10                        [10, 256, 28, 28]         --
├─Conv2d: 1-11                           [10, 512, 28, 28]         1,180,160
├─Conv2d: 1-12                           [10, 5

In [ ]:
# 计算LeNet5以及AlexNet各层的感受野的大小
!pip install torchinfo
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchinfo import summary

data = torch.ones(size=(10, 1, 32, 32))

class LeNet5(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)

  def forward(self, x):
    x = self.tanh(self.conv1(x))
    x = self.pool1(x)
    x = self.tanh(self.conv2(x))
    x = self.pool2(x)
    x = x.view(-1, 16*5*5)
    x = F.tanh(self.fc1(x))
    output = F.softmax(self.fc2(x), dim=1)
    output = F.softmax(x.view(-1, 16*5*5), dim=1)
    
net = LeNet5()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 计算卷积层中参数的数量
conv1 = nn.Conv2d(3, 6, 3)
conv2 = nn.Conv2d(6, 4, 3)

print(conv1.weight.numel())
print(conv1.bias.numel())
print(conv2.weight.numel())
print(conv2.bias.numel())

conv3 = nn.Conv2d(4, 16, 5, stride=2, padding=1) #(5*5*4)*16+16
conv4 = nn.Conv2d(16, 3, 5, stride=3, padding=2) #(5*5*16)*3+3
print(conv3.weight.numel())
print(conv4.weight.numel())

162
6
216
4
1600
1200


In [7]:
# 使用nn.Sequential
import torch
import torch.nn as nn
data = torch.ones(size=(10, 3, 229, 229))
# 不使用类，直接将需要串联的网络、函数等信息写在一个‘序列'里面
net = nn.Sequential(nn.Conv2d(3, 6, 3), nn.ReLU(inplace=True),
           nn.Conv2d(6, 4, 3), nn.ReLU(inplace=True),
           nn.MaxPool2d(2),
           nn.Conv2d(4, 16, 5, stride=2, padding=1), nn.ReLU(inplace=True),
           nn.Conv2d(16, 3, 5, stride=3, padding=2), nn.ReLU(inplace=True),
           nn.MaxPool2d(2))

net(data).shape

torch.Size([10, 3, 9, 9])

In [2]:
# 使用nn.Sequential复现VGG16
!pip install torchinfo
import torch
import torch.nn as nn
from torchinfo import summary
class VGG16(nn.Module):
  def __init__(self):
    super().__init__()
    self.features_ = nn.Sequential(nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    
                    nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    
                    nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    
                    nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    
                    nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
                    nn.MaxPool2d(2))
    
    self.clf_ = nn.Sequential(nn.Dropout(0.5),
                  nn.Linear(512*7*7, 4096), nn.ReLU(inplace=True),
                  nn.Dropout(0.5),
                  nn.Linear(4096, 4096), nn.ReLU(inplace=True),
                  nn.Linear(4096, 1000), nn.Softmax(dim=1))
  
  def forward(self, x):
    x = self.features_(x)
    x = x.view(-1, 512*7*7)
    output = self.clf_(x)
    return output

vgg = VGG16()
summary(vgg, input_size=(10, 3, 224, 224), device='cpu')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [10, 1000]                --
├─Sequential: 1-1                        [10, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [10, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [10, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [10, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [10, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [10, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [10, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [10, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [10, 128, 112, 112]       147,584
│    └─ReLU: 2-9                         [10, 128, 112, 112]       --
│    └─MaxPool2d: 2-10                   [10, 128, 56, 56]         --
│    └─Conv2d: 2-11                      [10, 256, 56, 56]         29

In [4]:
# NiN网络的复现
!pip install torchinfo
import torch 
import torch.nn as nn
from torchinfo import summary

data = torch.ones(size=(10, 3, 32, 32))

class NiN(nn.Module):
  def __init__(self):
    super().__init__()
    self.block1 = nn.Sequential(nn.Conv2d(3, 192, 5, padding=2), nn.ReLU(inplace=True),
                   nn.Conv2d(192, 160, 1), nn.ReLU(inplace=True),
                   nn.Conv2d(160, 96, 1), nn.ReLU(inplace=True),
                   nn.MaxPool2d(3, stride=2),
                   nn.Dropout(0.25))
    
    self.block2 = nn.Sequential(nn.Conv2d(96, 192, 5, padding=2), nn.ReLU(inplace=True),
                   nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
                   nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
                   nn.MaxPool2d(3, stride=2), 
                   nn.Dropout(0.25))
    
    self.block3 = nn.Sequential(nn.Conv2d(192, 192, 3, padding=1), nn.ReLU(inplace=True),
                   nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
                   nn.Conv2d(192, 10, 1), nn.ReLU(inplace=True),
                   nn.AvgPool2d(7, stride=1),
                   nn.Softmax(dim=1))
    
  def forward(self, x):
    output = self.block3(self.block2(self.block1(x)))
    return output

net = NiN()
net(data).shape
summary(net, input_size=(10, 3, 32, 32))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Layer (type:depth-idx)                   Output Shape              Param #
NiN                                      [10, 10, 1, 1]            --
├─Sequential: 1-1                        [10, 96, 15, 15]          --
│    └─Conv2d: 2-1                       [10, 192, 32, 32]         14,592
│    └─ReLU: 2-2                         [10, 192, 32, 32]         --
│    └─Conv2d: 2-3                       [10, 160, 32, 32]         30,880
│    └─ReLU: 2-4                         [10, 160, 32, 32]         --
│    └─Conv2d: 2-5                       [10, 96, 32, 32]          15,456
│    └─ReLU: 2-6                         [10, 96, 32, 32]          --
│    └─MaxPool2d: 2-7                    [10, 96, 15, 15]          --
│    └─Dropout: 2-8                      [10, 96, 15, 15]          --
├─Sequential: 1-2                        [10, 192, 7, 7]           --
│    └─Conv2d: 2-9                       [10, 192, 15, 15]         460,992
│    └─ReLU: 2-10                        [10, 192, 15, 15]         -